In [1]:
import pandas as pd
import numpy as np

In [2]:
Train =  pd.read_csv("Data/TrainEncoded.csv")
Test =  pd.read_csv("Data/TestEncoded.csv")

In [3]:
Train.head()

,Survived,1,2,3,0,1.1,2.1,3.1,4,5,...,E,F,G,T,0.2,1.3,"[0, 7.9)","[31, 38.9)",[38.9+),"[7.9, 14.4)"
0,0.0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,1.0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,1.0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,1.0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [4]:
Train.shape

(891, 40)

In [5]:
Test.head()

,1,2,3,0,1.1,2.1,3.1,4,5,8,...,E,F,G,T,0.2,1.3,"[0, 7.9)","[31, 38.9)",[38.9+),"[7.9, 14.4)"
0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [6]:
Test.shape

(418, 39)

In [7]:
class Outliers:
    def __init__(self, df):
        self.df = df
    
    ## function to find outliers limits
    def find_limits_and_delete_outliers(self, variable):

        IQR = self.df[variable].quantile(0.75) - self.df[variable].quantile(0.25)

        lower_limit = self.df[variable].quantile(0.25) - (IQR * 1.5)
        upper_limit = self.df[variable].quantile(0.75) + (IQR * 1.5)

        ## to find and delete values upper and under the outliers limits
        np.where(self.df[variable] > upper_limit)
        np.where(self.df[variable] < lower_limit)
        
        outliers = np.where(self.df[variable] > upper_limit, True, 
                            np.where(self.df[variable] < lower_limit, True, False))
        
        outliers_df = self.df.loc[outliers, variable]
        outliers_df_ids = self.df.loc[outliers]
        new_df = self.df.drop(outliers_df_ids.index).reset_index(drop=True)
        
        outliers_percentage = len(outliers_df) / len(self.df)
        
        return outliers_percentage, new_df

In [8]:
outliers_instance = Outliers(Train)

In [9]:
print(Train.columns.tolist())

['Survived', '1', '2', '3', '0', '1.1', '2.1', '3.1', '4', '5', '8', '0.1', '1.2', '2.2', '3.2', '4.1', '5.1', '6', '9', 'C', 'Q', 'S', '[0, 20)', '[20, 40)', '[40, 60)', '[60+)', 'A', 'B', 'C.1', 'D', 'E', 'F', 'G', 'T', '0.2', '1.3', '[0, 7.9)', '[31, 38.9)', '[38.9+)', '[7.9, 14.4)']


In [10]:
columns = ['Fare']

In [11]:
def removing_outliers(df, first_column, other_columns):
    outliers_instance = Outliers(df)
    outliers_percentage, new_df = outliers_instance.find_limits_and_delete_outliers(first_column)
    print(first_column, outliers_percentage, new_df.shape)
    
    for column in other_columns:
        outliers_instance = Outliers(new_df)
        outliers_percentage, new_df = outliers_instance.find_limits_and_delete_outliers(column)
        print(column, outliers_percentage, new_df.shape)
    return outliers_percentage, new_df

In [12]:
# outliers_percentage, new_df = removing_outliers(Train, columns[0], columns[1:])

KeyError: 'Fare'

In [ ]:
new_df.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
class Normalization:
    def __init__(self, df):
        self.df = df
        
    def normalization(self, columns):
        scaler = StandardScaler()

        ## features to normalice
        normalice = self.df[columns]
        scaler.fit(normalice)
        df_scaled = scaler.transform(normalice)
        
        new_columns = [str(x)+'_scaled' for x in columns]
        
        df_scaled = pd.DataFrame(df_scaled, columns=new_columns)
        self.df = pd.concat([self.df,df_scaled], axis=1)
        
        self.df = self.df.drop(columns, axis=1)
        return self.df
    
    # Final dataframe
    def final_dataframe(self):
        return self.df

In [ ]:
norm_instance_train = Normalization(new_df)

In [ ]:
print(new_df.columns.tolist())

In [ ]:
columns = ['Fare']

In [ ]:
norm_instance_train.normalization(columns).head()

In [ ]:
norm_instance_train.final_dataframe().head()

In [ ]:
norm_instance_test = Normalization(Test)

In [ ]:
norm_instance_test.normalization(columns).head()

In [ ]:
norm_instance_train.final_dataframe().to_csv("Data/TrainOutliersNormalization.csv", index=False)
norm_instance_test.final_dataframe().to_csv("Data/TestOutliersNormalization.csv", index=False)